# CAO Points Notebook
***

This Jupyter notebook will contain a clear and concise overview of how to load CAO points information from the
CAO website into a pandas data frame, a detailed comparison of CAO points in 2019, 2020, and 2021 and some visualisations to enhance the viewer experience.

<br>

### Required Modules
***

In [1]:
# Convenient HTTP requests
import requests as rq # Requests

# Regular Expressions
import re # Regular Expressions

# Dates and times
import datetime as dt

# Pandas for dataframes
import pandas as pd

# For downloading files
import urllib.request as urlrq

# Numpy for numerical operations
import numpy as np

# Pyplot for plots
import matplotlib.pyplot as plt

<br>

***
## Section 1: Loading the CAO Points into a Pandas Dataframe  
***

### 2021 Level 8 Points
[https://www.cao.ie/index.php?page=points&p=2021&bb=points](https://www.cao.ie/index.php?page=points&p=2021&bb=points)
***

In [2]:
# Fetch the CAO URL
resp = rq.get('http://www2.cao.ie/points/l8.php')
# Testing this request - we want to get 200 back to confirm everything is OK
resp

<Response [200]>

In [3]:
# Get the current date and time
now = dt.datetime.now()

# Save the date and time as a string so it can be reused
now_str = now.strftime('%Y%m%d_%H%M%S')

# Create a file path for the original data using the date and time
path21 = 'data/cao2021_' + now_str + '.html'

In [4]:
# Server uses the wrong encoding, we need to change it from iso-8859-1 to cp1252
original_encoding = resp.encoding
original_encoding
resp.encoding = 'cp1252'

In [5]:
# Save the file
with open(path21, 'w') as f:
    f.write(resp.text)

In [6]:
# Compile the regular expression for matching only lines with course info
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.{1,53})   +(.{0,5})  +(.{0,5})')

In [7]:
# Loop through the lines of the response content
no_lines = 0
for line in resp.iter_lines():
    if re_course.match(line.decode('iso-8859-1')):
        #print(line)
        no_lines = no_lines +1
      
# Check how many lines are being captured - manual checks showed this should be 949
print (no_lines)

949


In [8]:
csv_path = 'cao2021_re_' + now_str + '.csv'
with open(csv_path,'w') as f:
    
    # Now loop through and print out the lines matching the RE
    for line in resp.iter_lines():
        if re_course.match(line.decode('cp1252')):
            csv_version = re_course.sub(r'\1,\2,\3,\4', line.decode('cp1252')).strip()
            #print(csv_version)
            #print(line)
            f.write(csv_version + '\n')

In [9]:
# Create a dataframe for the 2021 data with column names
df21 = pd.read_csv(csv_path, header=None, names=["Course_Code", "Course_Title", "Points_R1", "Points_R2"])

# Show the first 5 rows of the new dataframe
df21.head()

,Course_Code,Course_Title,Points_R1,Points_R2
0,AL801,Software Design for Virtual Reality and Gaming...,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructur...,321,NaN
4,AL810,Quantity Surveying ...,328,NaN


<br>

## 2020 Level 8 Points
[https://www.cao.ie/index.php?page=points&p=2020&bb=points](https://www.cao.ie/index.php?page=points&p=2020&bb=points)
***

In [10]:
# Create a filepath for the 2020 points file
path20 = 'data/cao2020_' + now_str + '.xlsx'

# Save original data file to disk
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx',path20)

('data/cao2020_20220101_204029.xlsx',
 <http.client.HTTPMessage at 0x1c1812849d0>)

In [11]:
# Download and parse the excel spreadsheet into a dataframe
df20 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx',skiprows=10)

# Show the first 5 rows of the dataframe
df20.head()

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Create a filepath for the pandas dataframe
dfpath20 = 'data/cao2020_df' + now_str + '.csv'

# Save the dataframe
df20.to_csv(dfpath20)

<br>

## 2019 Level 8 Points
[http://www.cao.ie/index.php?page=points&p=2019](http://www.cao.ie/index.php?page=points&p=2019)
***

##### Steps to Reproduce Data
1. Download pdf from the CAO website (see link above)  
2. Open pdf in Microsoft Word  
3. Save file in .docx format  
4. Save another copy of the word doc for editing  
5. Delete headers and footers  
6. Delete preamble on first page  
7. Select all and copy  
8. Paste into Notepad++  
9. Cut HEI names and paste onto beginning of each applicable course line, followed by a tab  
10. Delete blank lines
11. Replace double tabs with a single tab  
12. Change backticks to apostrophes  

In [13]:
# Read in the data
df19 = pd.read_csv("data/cao2019_03112021.csv", sep='\t')

# Show the first 5 rows
df19.head()

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0


<br>

#### Using Concat & Duplicate on the Dataframes
***

In [14]:
# Create a dataframe for 2021 using just the course code and title columns
courses21 = df21[["Course_Code", "Course_Title"]]

# Take a quick peak
courses21.head()

,Course_Code,Course_Title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...


In [15]:
# Doing the same thing for 2020
courses20 = df20[["COURSE CODE2", "COURSE TITLE"]]

# Change the column names to match the 2021 df
courses20.columns = ["Course_Code", "Course_Title"]

# Take a look
courses20.head()

,Course_Code,Course_Title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...


In [16]:
# Aaaaaaand same again for 2019
courses19 = df19[["Course Code", "INSTITUTION and COURSE"]]

# Change the colum names to match the 2021 df
courses19.columns = ["Course_Code", "Course_Title"]

# What does that look like?
courses19.head()

,Course_Code,Course_Title
0,AL801,Software Design with Virtual Reality and Gaming
1,AL802,Software Design with Cloud Computing
2,AL803,Software Design with Mobile Apps and Connected...
3,AL805,Network Management and Cloud Infrastructure
4,AL810,Quantity Surveying


In [17]:
# Combine the 3 dfs into 1 with concat
all_courses = pd.concat([courses21, courses20, courses19], ignore_index=True)
all_courses.head()

,Course_Code,Course_Title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...


In [18]:
# How many courses are duplicated?
all_courses[all_courses.duplicated()].count()

Course_Code     679
Course_Title    679
dtype: int64

In [19]:
# Returns a copy of the df with duplicates removed
all_courses.drop_duplicates(ignore_index=True)

,Course_Code,Course_Title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...
...,...,...
2659,TL802,"TV, Radio and New Media"
2660,TL803,Music Technology
2661,TL812,Computing with Digital Media
2662,TL842,Construction Management


In [20]:
# Return all courses where course code was duplicated (i.e. course was available each year)
all_courses[all_courses.duplicated(subset=["Course_Code"])]

,Course_Code,Course_Title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [21]:
# Store this in a varibale for analysis later
courses_each_year = all_courses[all_courses.duplicated(subset=["Course_Code"])]

In [22]:
# Using sort will show that the course names varied slightly year on year
all_courses.sort_values('Course_Code')

,Course_Code,Course_Title
175,AC120,International Business ...
949,AC120,International Business
2581,AC120,International Business
950,AC137,Liberal Arts
2582,AC137,Liberal Arts
...,...,...
2412,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering ...
3342,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education ...


<br>

#### Using Join Method on the Dataframes
***

In [23]:
# Set course code as the index
all_courses.set_index("Course_Code",inplace=True)

# Set course code as index & tweak column names on df21
df21.columns = ["Course_Code", "Course_Title", "2021_Points_R1", "2021_Points_R2"]
df21.set_index("Course_Code",inplace=True)

In [24]:
# Creating new dataframe for all courses by joining existing df to column 2021_Points_R1
df_allcourses = all_courses.join(df21[["2021_Points_R1","2021_Points_R2"]])
df_allcourses.head()

,Course_Title,2021_Points_R1,2021_Points_R2
Course_Code,,,
AC120,International Business ...,294,294
AC120,International Business,294,294
AC120,International Business,294,294
AC137,Liberal Arts ...,271,270
AC137,Liberal Arts,271,270


In [25]:
# Columns are messy in df20 - make a new df for 2020 points with just course code, R1 & R2 Points
df20_r1r2=df20[["COURSE CODE2", "R1 POINTS", "R2 POINTS"]]

# Change the column names to match df21
df20_r1r2.columns = ["Course_Code","2020_Points_R1", "2020_Points_R2"]
df20_r1r2.head()

,Course_Code,2020_Points_R1,2020_Points_R2
0,AC120,209,NaN
1,AC137,252,NaN
2,AD101,#+matric,NaN
3,AD102,#+matric,NaN
4,AD103,#+matric,NaN


In [26]:
# Set Course Code as the index
df20_r1r2.set_index("Course_Code",inplace=True)

In [27]:
# Join to the df20_r1 df
df_allcourses = df_allcourses.join(df20_r1r2)
df_allcourses.head()

,Course_Title,2021_Points_R1,2021_Points_R2,2020_Points_R1,2020_Points_R2
Course_Code,,,,,
AC120,International Business ...,294,294,209,NaN
AC120,International Business,294,294,209,NaN
AC120,International Business,294,294,209,NaN
AC137,Liberal Arts ...,271,270,252,NaN
AC137,Liberal Arts,271,270,252,NaN


In [28]:
# Doing the same for 2019 points - making a new df with just course code. R1 & R2 points
df19_r1r2=df19[["Course Code", "Mid", "EOS"]] # Mid looks like it was the R1 Points, EOS R2

# Change the column names to match df21
df19_r1r2.columns = ["Course_Code","2019_Points_R1", "2019_Points_R2"]
df19_r1r2.head()

,Course_Code,2019_Points_R1,2019_Points_R2
0,AL801,328.0,304
1,AL802,306.0,301
2,AL803,337.0,309
3,AL805,442.0,329
4,AL810,349.0,307


In [29]:
# Setting course code as the index
df19_r1r2.set_index("Course_Code",inplace=True)

In [30]:
# Join to the df20_r1 df
df_allcourses = df_allcourses.join(df19_r1r2)

# Now we can have a look at the new df with R1 points from each of the 3 years
df_allcourses.head()

,Course_Title,2021_Points_R1,2021_Points_R2,2020_Points_R1,2020_Points_R2,2019_Points_R1,2019_Points_R2
Course_Code,,,,,,,
AC120,International Business ...,294,294,209,NaN,269.0,234
AC120,International Business,294,294,209,NaN,269.0,234
AC120,International Business,294,294,209,NaN,269.0,234
AC137,Liberal Arts ...,271,270,252,NaN,275.0,252
AC137,Liberal Arts,271,270,252,NaN,275.0,252


<br>

***
## Section 2: Comparing CAO Points in 2019, 2020 & 2021
***

**2020 Points Analysis**  
Let's start by analysing the 2020 points.  

In [31]:
df_allcourses['2021_Points_R1'].describe()

count      2547
unique      377
top       300  
freq         49
Name: 2021_Points_R1, dtype: object

In [32]:
df_allcourses.describe()

,2019_Points_R1
count,2437.000000
mean,428.504309
std,130.640140
min,57.000000
25%,345.000000
50%,401.000000
75%,487.000000
max,1073.000000


In [33]:
# Sorting by course code
df_allcourses = df_allcourses.sort_values("Course_Code")

# Removing duplicates by every column except course title (as this varied over the years), keeping first iteration
df_allcourses = df_allcourses.drop_duplicates(subset = ("2021_Points_R1", "2021_Points_R2", "2020_Points_R1", 
                              "2020_Points_R2", "2019_Points_R1", "2019_Points_R2"), keep = "first", ignore_index=False)

In [34]:
df_allcourses.describe()

,2019_Points_R1
count,914.000000
mean,426.114880
std,132.906752
min,57.000000
25%,339.000000
50%,395.000000
75%,477.000000
max,1073.000000


In [35]:
# Create a filepath for the pandas dataframe
savepath = 'df_allcourses.csv'

# Save the dataframe
df_allcourses.to_csv(savepath)

<br>

### Section 3: Creating a Function to Perform the Analysis
***

In [58]:
def analysis():
    # Reading in the data from the csv file created above
    df_allcourses = pd.read_csv(savepath, index_col=0) 
    print ("ANALYSING CAO POINTS")
    print ("\n")
    print ("*****\nNUMBER OF COURSES AVAILABLE IN EACH YEAR\n*****")
    print("2021:", df_allcourses.value_counts("2021_Points_R1").sum())
    print("2020:", df_allcourses.value_counts("2020_Points_R1").sum())
    print("2019:", df_allcourses.value_counts("2019_Points_R1").sum())
    print ("\n")
    print ("*****\nAQA (ALL QUALIFIED APPLICANTS)\n*****")
    print ("Here are the courses where All Qualified Applicants were offered a place in 2021 & 2020:")
    print ("\n")
    print ("2021:")
    print (df_allcourses.loc[df_allcourses["2021_Points_R2"] == "AQA", "Course_Title"])
    print ("\n")
    print ("2020:")
    print (df_allcourses.loc[df_allcourses["2020_Points_R1"] == "AQA", "Course_Title"])
    print ("\n")
    print ("NOTE: no AQA values were observed in the 2019 points data.")
    
    # Telling pandas to not diplay decimal places 
    # found at https://newbedev.com/pandas-converting-floats-to-strings-without-decimals
    pd.options.display.float_format = '{:,.0f}'.format
    
    # Converting all columns to float
    df_allcourses["2021_Points_R1"] = pd.to_numeric(df_allcourses["2021_Points_R1"], errors = "coerce").fillna(np.nan)
    df_allcourses["2021_Points_R2"] = pd.to_numeric(df_allcourses["2021_Points_R2"], errors = "coerce").fillna(np.nan)
    df_allcourses["2020_Points_R1"] = pd.to_numeric(df_allcourses["2020_Points_R1"], errors = "coerce").fillna(np.nan)
    df_allcourses["2020_Points_R2"] = pd.to_numeric(df_allcourses["2020_Points_R2"], errors = "coerce").fillna(np.nan)
    df_allcourses["2019_Points_R1"] = pd.to_numeric(df_allcourses["2019_Points_R1"], errors = "coerce").fillna(np.nan)
    df_allcourses["2019_Points_R2"] = pd.to_numeric(df_allcourses["2019_Points_R2"], errors = "coerce").fillna(np.nan)
    
    # Creating variables for min, max, mean, & median points in each year
    max2021 = df_allcourses['2021_Points_R1'].max()
    min2021 = df_allcourses['2021_Points_R2'].min()
    max2020 = df_allcourses['2020_Points_R1'].max()
    min2020 = df_allcourses['2020_Points_R2'].min()
    max2019 = df_allcourses['2019_Points_R1'].max()
    min2019 = df_allcourses['2019_Points_R2'].min()

    # Rounding the mean values as I was getting values with several decimal places in testing
    mean2021 = round(df_allcourses['2021_Points_R1'].mean(),2)
    mean2020 = round(df_allcourses['2020_Points_R1'].mean(),2)
    mean2019 = round(df_allcourses['2019_Points_R1'].mean(),2)

    median2021 = df_allcourses['2021_Points_R1'].median()
    median2020 = df_allcourses['2020_Points_R1'].median()
    median2019 = df_allcourses['2019_Points_R1'].median()

    # I had also calculated the mode but found this was not of any use due to NaN values
    # modevalues = df_allcourses.mode(axis = 0)
    
    print ("\n")
    print ("*****\nGENERAL STATISTICS\n*****")
    print (df_allcourses.describe())
    print ("\n")
    print ("*****\nMEAN & MEDIAN POINTS - ROUND 1 \n*****")
    print ("MEAN \n2021:", mean2021,"\t2020:", mean2020, "\t2019:", mean2019)
    print ("\n")
    print ("MEDIAN \n2021:", median2021, "\t2020:", median2020, "\t2019:", median2019)
    print ("\n")
    print ("*****\nCOURSES WITH HIGHEST & LOWEST POINTS\n*****")
    print ("2021: Highest Points")
    print (df_allcourses.loc[df_allcourses["2021_Points_R1"] == max2021, "Course_Title"])
    print ("Points:", max2021)
    print ("\n")
    print ("2021: Lowest Points")
    print (df_allcourses.loc[df_allcourses["2021_Points_R2"] == min2021, "Course_Title"])
    print ("Points:", min2021)
    print ("\n")
    print ("2020: Highest Points")
    print (df_allcourses.loc[df_allcourses["2020_Points_R1"] == max2020, "Course_Title"])
    print ("Points:", max2020)
    print ("\n")
    print ("2020: Lowest Points")
    print (df_allcourses.loc[df_allcourses["2020_Points_R2"] == min2020, "Course_Title"])
    print ("Points:", min2020)
    print ("\n")
    print ("2019: Highest Points")
    print (df_allcourses.loc[df_allcourses["2019_Points_R1"] == max2019, "Course_Title"])
    print ("Points:", max2019)
    print ("\n")
    print ("2019: Lowest Points")
    print (df_allcourses.loc[df_allcourses["2019_Points_R2"] == min2019, "Course_Title"])
    print ("Points:", min2019)
    print ("\n")
    
    # Dropping NaNs at this point as these can't be compared against each other
    df_allcourses = df_allcourses.dropna()
    
    # Redefining variables for min & max points in each year with NaN removed
    max2021 = df_allcourses['2021_Points_R1'].max()
    min2021 = df_allcourses['2021_Points_R2'].min()
    max2020 = df_allcourses['2020_Points_R1'].max()
    min2020 = df_allcourses['2020_Points_R2'].min()
    max2019 = df_allcourses['2019_Points_R1'].max()
    min2019 = df_allcourses['2019_Points_R2'].min()
    
    print ("NOTE: as the dataframe contains NaN values, it is difficult to compare all courses across the three years.")
    print ("If we were to remove the NaN values, these would be the courses with the highest and lowest points:")
    
    print ("\n")
    print ("2021: Highest Points")
    print (df_allcourses.loc[df_allcourses["2021_Points_R1"] == max2021, "Course_Title"])
    print ("Points:", max2021)
    print ("\n")
    print ("2021: Lowest Points")
    print (df_allcourses.loc[df_allcourses["2021_Points_R2"] == min2021, "Course_Title"])
    print ("Points:", min2021)
    print ("\n")
    print ("2020: Highest Points")
    print (df_allcourses.loc[df_allcourses["2020_Points_R1"] == max2020, "Course_Title"])
    print ("Points:", max2020)
    print ("\n")
    print ("2020: Lowest Points")
    print (df_allcourses.loc[df_allcourses["2020_Points_R2"] == min2020, "Course_Title"])
    print ("Points:", min2020)
    print ("\n")
    print ("2019: Highest Points")
    print (df_allcourses.loc[df_allcourses["2019_Points_R1"] == max2019, "Course_Title"])
    print ("Points:", max2019)
    print ("\n")
    print ("2019: Lowest Points")
    print (df_allcourses.loc[df_allcourses["2019_Points_R2"] == min2019, "Course_Title"])
    print ("Points:", min2019)
    print ("\n")
    print("The above shows an increase in the points required for the course with the highest points\
    TR019: Law and German, since 2019, when the points required were", max2019,"\nThe points fell to", max2020,"in 2020 but\
    again increased to",max2021,"in 2021.")

    return print ("Analysis Complete")

In [59]:
analysis()

ANALYSING CAO POINTS


*****
NUMBER OF COURSES AVAILABLE IN EACH YEAR
*****
2021: 923
2020: 1181
2019: 914


*****
AQA (ALL QUALIFIED APPLICANTS)
*****
Here are the courses where All Qualified Applicants were offered a place in 2021 & 2020:


2021:
Course_Code
MH801    Early Childhood - Teaching and Learning (part-...
Name: Course_Title, dtype: object


2020:
Course_Code
AL600                                      Software Design
AL601                                 Computer Engineering
AL604                                    Civil Engineering
TU772           Business (Common Entry with Award options)
WD003                                             Business
WD152    Visual Art                                    ...
WD200                                       Arts (options)
Name: Course_Title, dtype: object


NOTE: no AQA values were observed in the 2019 points data.


*****
GENERAL STATISTICS
*****
       2021_Points_R1  2021_Points_R2  2020_Points_R1  2020_Points_R2  \
count       

<br>

#### AQA

In [ ]:
print ("Here are the courses where All Qualified Applicants were offered a place in 2021 & 2020.")
print ("\n")
print ("2021:")
print (df_allcourses.loc[df_allcourses["2021_Points_R2"] == "AQA", "Course_Title"])
print ("\n")
print ("2020:")
print (df_allcourses.loc[df_allcourses["2020_Points_R1"] == "AQA", "Course_Title"])
print ("\n")
print ("Note: no AQA values were observed in the 2019 points data.")

<br>

In [ ]:
# Checking the column types before moving onto further analysis
df_allcourses.dtypes

In [ ]:
# Telling pandas to not diplay decimal places 
# found at https://newbedev.com/pandas-converting-floats-to-strings-without-decimals
pd.options.display.float_format = '{:,.0f}'.format

In [ ]:
# Converting the 2019_Points_R1 column from float to String 
# adapted from https://www.geeksforgeeks.org/how-to-convert-floats-to-strings-in-pandas-dataframe/
# df_allcourses = df_allcourses.replace('\.0', '', regex=True)

In [ ]:
# Converting all columns to float
df_allcourses["2021_Points_R1"] = pd.to_numeric(df_allcourses["2021_Points_R1"], errors = "coerce").fillna(np.nan)
df_allcourses["2021_Points_R2"] = pd.to_numeric(df_allcourses["2021_Points_R2"], errors = "coerce").fillna(np.nan)
df_allcourses["2020_Points_R1"] = pd.to_numeric(df_allcourses["2020_Points_R1"], errors = "coerce").fillna(np.nan)
df_allcourses["2020_Points_R2"] = pd.to_numeric(df_allcourses["2020_Points_R2"], errors = "coerce").fillna(np.nan)
df_allcourses["2019_Points_R1"] = pd.to_numeric(df_allcourses["2019_Points_R1"], errors = "coerce").fillna(np.nan)
df_allcourses["2019_Points_R2"] = pd.to_numeric(df_allcourses["2019_Points_R2"], errors = "coerce").fillna(np.nan)

In [ ]:
df_allcourses.dtypes

In [ ]:
df_allcourses.head()

In [ ]:
print ("Courses available in each year:")
print("2021:", df_allcourses.value_counts("2021_Points_R1").sum())
print("2020:", df_allcourses.value_counts("2020_Points_R1").sum())
print("2019:", df_allcourses.value_counts("2019_Points_R1").sum())

In [ ]:
# Dropping NaNs as these can't be compared against each other
df_allcourses = df_allcourses.dropna()

In [ ]:
# https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
# df.drop(df.loc[df['line_race']==0].index, inplace=True)

In [ ]:
# Creating variables for min, max, mean, mode & median points in each year
max2021 = df_allcourses['2021_Points_R1'].max()
min2021 = df_allcourses['2021_Points_R2'].min()
max2020 = df_allcourses['2020_Points_R1'].max()
min2020 = df_allcourses['2020_Points_R2'].min()
max2019 = df_allcourses['2019_Points_R1'].max()
min2019 = df_allcourses['2019_Points_R2'].min()

mean2021 = df_allcourses.groupby('2021_Points_R1').mean()
mean2020 = df_allcourses.groupby('2020_Points_R1').mean()
mean2019 = df_allcourses.groupby('2019_Points_R1').mean()

median2021 = df_allcourses.groupby('2021_Points_R1').median()
median2020 = df_allcourses.groupby('2020_Points_R1').median()
median2019 = df_allcourses.groupby('2019_Points_R1').median()

modevalues = df_allcourses.mode(axis = 0)

<br>

### Minimum & Maximum Points by Year

Here are the courses with the highest and lowest points for each year.  

##### 2021: Highest & Lowest Points  

In [ ]:
print ("2021: Highest Points")
print (df_allcourses.loc[df_allcourses["2021_Points_R1"] == max2021, "Course_Title"])
print ("Points:", max2021)
print ("\n")
print ("2021: Lowest Points")
print (df_allcourses.loc[df_allcourses["2021_Points_R2"] == min2021, "Course_Title"])
print ("Points:", min2021)

<br>

##### 2020: Highest & Lowest Points  

In [ ]:
print ("2020: Highest Points")
print (df_allcourses.loc[df_allcourses["2020_Points_R1"] == max2020, "Course_Title"])
print ("Points:", max2020)
print ("\n")
print ("2020: Lowest Points")
print (df_allcourses.loc[df_allcourses["2020_Points_R2"] == min2020, "Course_Title"])
print ("Points:", min2020)

<br>

##### 2019 Highest & Lowest Points

In [ ]:
print ("2019: Highest Points")
print (df_allcourses.loc[df_allcourses["2019_Points_R1"] == max2019, "Course_Title"])
print ("Points:", max2019)
print ("\n")
print ("2019: Lowest Points")
print (df_allcourses.loc[df_allcourses["2019_Points_R2"] == min2019, "Course_Title"])
print ("Points:", min2019)

<br>

##### Mean, Mode & Median Points

<br>

### Trends

<br>

### References
***

[] https://datatofish.com/dropna/  

[] https://www.geeksforgeeks.org/how-to-convert-floats-to-strings-in-pandas-dataframe/  

[] https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/  

[] https://www.kite.com/python/answers/how-to-replace-column-values-in-a-pandas-dataframe-in-python  

[] https://newbedev.com/pandas-converting-floats-to-strings-without-decimals  

[] https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html  

[] https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value  